# Predicted vs true predation matrices

### Step 1: Simulate True vs Predicted Interactions

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Simulated species interaction pairs
data = {
    'con.taxonomy': ['Wolf', 'Fox', 'Wolf', 'Deer', 'Eagle', 'Frog'],
    'res.taxonomy': ['Deer', 'Rabbit', 'Rabbit', 'Grass', 'Mouse', 'Fly'],
    'true_label':    [1, 1, 0, 0, 1, 0],  # True interaction labels
    'pred_label':    [1, 0, 1, 0, 1, 0]   # Predicted labels from model
}

df = pd.DataFrame(data)

### Step 2: Extract Confusion Matrix Components

In [ ]:
# Compute confusion matrix
tn, fp, fn, tp = confusion_matrix(df['true_label'], df['pred_label']).ravel()

print("Confusion Matrix Breakdown:")
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")

### Step 3: Construct and View the Predation Matrix

In [ ]:
# Filter only predicted links
edges = df[df['pred_label'] == 1]

# Create binary predation matrix
predation_matrix = edges.assign(value=1).pivot_table(
    index='con.taxonomy',
    columns='res.taxonomy',
    values='value',
    fill_value=0
)

print("\nPredation Matrix (Predicted Interactions):")
predation_matrix

#  Layout predation matrix visualizations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# -----------------------------
# Fake species and body mass
species = ['A', 'B', 'C', 'D', 'E', 'F']
body_mass = [0.1, 0.5, 1.0, 3.0, 6.0, 10.0]
species_df = pd.DataFrame({'species': species, 'body_mass': body_mass}).sort_values(by='body_mass')

# True and predicted interactions
true_links = [('A', 'C'), ('B', 'D'), ('C', 'F'), ('D', 'F'), ('B', 'E')]
pred_links = [('A', 'C'), ('B', 'E'), ('C', 'F'), ('E', 'F'), ('A', 'D')]

# Sort species by body mass
species_sorted = species_df['species'].values

# -----------------------------
# SETUP: Matplotlib with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# --- Subplot 1: Predation Matrix (Empirical vs Predicted) ---

# Plot predicted links (open circles)
for prey_i, prey_sp in enumerate(species_sorted):
    for pred_j, pred_sp in enumerate(species_sorted):
        if (prey_sp, pred_sp) in pred_links:
            ax1.plot(pred_j, prey_i, 'o', markerfacecolor='none', markeredgecolor='black', markersize=10)

# Plot empirical links (filled circles)
for prey_i, prey_sp in enumerate(species_sorted):
    for pred_j, pred_sp in enumerate(species_sorted):
        if (prey_sp, pred_sp) in true_links:
            ax1.plot(pred_j, prey_i, 'o', markerfacecolor='black', markeredgecolor='black', markersize=5)

# Axis settings
ax1.set_xticks(range(len(species_sorted)))
ax1.set_xticklabels(species_sorted, rotation=90)
ax1.set_yticks(range(len(species_sorted)))
ax1.set_yticklabels(species_sorted)
ax1.set_xlabel("Predators")
ax1.set_ylabel("Prey")
ax1.set_title("Predation Matrix")
ax1.invert_yaxis()
ax1.grid(False)

# --- Subplot 2: Network View ---

# Build graph
G = nx.DiGraph()
for sp, mass in zip(species_df['species'], species_df['body_mass']):
    G.add_node(sp, mass=mass)

# Classify and add edges
for link in set(true_links + pred_links):
    if link in true_links and link in pred_links:
        G.add_edge(*link, label='TP')
    elif link in true_links:
        G.add_edge(*link, label='FN')
    elif link in pred_links:
        G.add_edge(*link, label='FP')

# Layout
pos = nx.spring_layout(G, seed=42)

# Node sizes by body mass
node_sizes = [species_df.loc[species_df['species'] == n, 'body_mass'].values[0] * 300 for n in G.nodes]
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='white', edgecolors='black', ax=ax2)
nx.draw_networkx_labels(G, pos, font_size=10, ax=ax2)

# Edge styling
edge_colors, edge_styles = [], []
for u, v, data in G.edges(data=True):
    if data['label'] == 'TP':
        edge_colors.append('green')
        edge_styles.append('solid')
    elif data['label'] == 'FP':
        edge_colors.append('gray')
        edge_styles.append('dashed')
    elif data['label'] == 'FN':
        edge_colors.append('red')
        edge_styles.append('solid')

# Draw edges
for (u, v), color, style in zip(G.edges(), edge_colors, edge_styles):
    nx.draw_networkx_edges(G, pos, edgelist=[(u, v)],
                           edge_color=color, style=style,
                           arrows=True, arrowsize=15, width=2, ax=ax2)

# Final touches
ax2.set_title("Network View")
ax2.axis('off')

# --- Layout and show ---
plt.tight_layout()
plt.show()

#  Layout predation matrix visualizations - lakes_ecosystem

In [ ]:
# Re-import necessary packages after kernel reset
import pandas as pd
import matplotlib.pyplot as plt

# Reload CSV files
tp_links = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/testing/lakes/TP_links_exp_1_K_10.csv", header=None).values
fp_links = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/testing/lakes/FP_links_exp_1_K_10.csv", header=None).values
fn_links = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/testing/lakes/FN_links_exp_1_K_10.csv", header=None).values

# Build node index
all_nodes = set(tp_links.flatten()).union(fp_links.flatten()).union(fn_links.flatten())
node_list = sorted(all_nodes)
node_index = {node: idx for idx, node in enumerate(node_list)}
N = len(node_list)

# Fill matrix with tag values
matrix = [['' for _ in range(N)] for _ in range(N)]
for i, j in tp_links:
    matrix[node_index[i]][node_index[j]] = 'TP'
for i, j in fp_links:
    matrix[node_index[i]][node_index[j]] = 'FP'
for i, j in fn_links:
    matrix[node_index[i]][node_index[j]] = 'FN'

# Plot layout predation matrix
fig, ax = plt.subplots(figsize=(12, 12))
for i in range(N):
    for j in range(N):
        tag = matrix[i][j]
        if tag == 'TP':
            ax.plot(j, i, 'o', markerfacecolor='black', markeredgecolor='black', markersize=6, label='TP' if i == j == 0 else "")
        elif tag == 'FP':
            ax.plot(j, i, 'o', markerfacecolor='none', markeredgecolor='gray', markersize=8, label='FP' if i == j == 0 else "")
        elif tag == 'FN':
            ax.plot(j, i, 'o', markerfacecolor='red', markeredgecolor='red', markersize=4, label='FN' if i == j == 0 else "")

ax.set_xticks(range(N))
ax.set_yticks(range(N))
ax.set_xticklabels(node_list, rotation=90, fontsize=6)
ax.set_yticklabels(node_list, fontsize=6)
ax.set_xlabel("Predator (j)")
ax.set_ylabel("Prey (i)")
ax.set_title("Predation Matrix: True vs Predicted Interactions")
ax.invert_yaxis()
ax.grid(False)

ax.plot([], [], 'o', markerfacecolor='black', markeredgecolor='black', markersize=6, label='TP')
ax.plot([], [], 'o', markerfacecolor='none', markeredgecolor='gray', markersize=8, label='FP')
ax.plot([], [], 'o', markerfacecolor='red', markeredgecolor='red', markersize=4, label='FN')

ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Re-import necessary packages after kernel reset
import pandas as pd
import matplotlib.pyplot as plt

# Reload CSV files
tp_links = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/testing/Chesapeake Bay/TP_links_exp_1_K_10.csv", header=None).values
fp_links = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/testing/Chesapeake Bay/FP_links_exp_1_K_10.csv", header=None).values
fn_links = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/testing/Chesapeake Bay/FN_links_exp_1_K_10.csv", header=None).values

# Build node index
all_nodes = set(tp_links.flatten()).union(fp_links.flatten()).union(fn_links.flatten())
node_list = sorted(all_nodes)
node_index = {node: idx for idx, node in enumerate(node_list)}
N = len(node_list)

# Fill matrix with tag values
matrix = [['' for _ in range(N)] for _ in range(N)]
for i, j in tp_links:
    matrix[node_index[i]][node_index[j]] = 'TP'
for i, j in fp_links:
    matrix[node_index[i]][node_index[j]] = 'FP'
for i, j in fn_links:
    matrix[node_index[i]][node_index[j]] = 'FN'

# Plot layout predation matrix
fig, ax = plt.subplots(figsize=(12, 12))
for i in range(N):
    for j in range(N):
        tag = matrix[i][j]
        if tag == 'TP':
            ax.plot(j, i, 'o', markerfacecolor='black', markeredgecolor='black', markersize=6, label='TP' if i == j == 0 else "")
        elif tag == 'FP':
            ax.plot(j, i, 'o', markerfacecolor='none', markeredgecolor='gray', markersize=8, label='FP' if i == j == 0 else "")
        elif tag == 'FN':
            ax.plot(j, i, 'o', markerfacecolor='red', markeredgecolor='red', markersize=4, label='FN' if i == j == 0 else "")

ax.set_xticks(range(N))
ax.set_yticks(range(N))
ax.set_xticklabels(node_list, rotation=90, fontsize=6)
ax.set_yticklabels(node_list, fontsize=6)
ax.set_xlabel("Predator (j)")
ax.set_ylabel("Prey (i)")
ax.set_title("Predation Matrix: True vs Predicted Interactions")
ax.invert_yaxis()
ax.grid(False)

ax.plot([], [], 'o', markerfacecolor='black', markeredgecolor='black', markersize=6, label='TP')
ax.plot([], [], 'o', markerfacecolor='none', markeredgecolor='gray', markersize=8, label='FP')
ax.plot([], [], 'o', markerfacecolor='red', markeredgecolor='red', markersize=4, label='FN')

ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Re-import libraries after kernel reset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reload the food web CSV file
file_path = "/Users/jorge/Desktop/PhD/Code/ExtractFoodWebs/foodwebs_mat_by_ecosystem/CSV/lakes_foodweb.csv"
df = pd.read_csv(file_path)

# Extract prey and predator taxonomy columns
prey = df['res.taxonomy'].astype(str)
predator = df['con.taxonomy'].astype(str)

# Create a sorted list of all unique species
species = sorted(set(prey).union(set(predator)))
species_index = {name: i for i, name in enumerate(species)}
N = len(species)

# Create an adjacency matrix (binary)
adj_matrix = np.zeros((N, N), dtype=int)
for res, con in zip(prey, predator):
    i = species_index[res]
    j = species_index[con]
    adj_matrix[i, j] = 1

# Plot the adjacency matrix
fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(adj_matrix, cmap='Greys', interpolation='none')

ax.set_xticks(range(N))
ax.set_yticks(range(N))
ax.set_xticklabels(species, rotation=90, fontsize=4)
ax.set_yticklabels(species, fontsize=4)
ax.set_xlabel("Predator (con.taxonomy)")
ax.set_ylabel("Prey (res.taxonomy)")
ax.set_title("Adjacency Matrix from Food Web (res.taxonomy vs con.taxonomy)")
ax.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load CSV
file_path = "/Users/jorge/Desktop/PhD/Code/ExtractFoodWebs/foodwebs_csv/SF1M2.csv"
df = pd.read_csv(file_path, )

# Extract relevant columns
df['res.taxonomy'] = df['res.taxonomy'].astype(str)
df['con.taxonomy'] = df['con.taxonomy'].astype(str)

# Combine species and mass from both roles
res_masses = df[['res.taxonomy', 'res.mass.mean.g.']].dropna().rename(
    columns={'res.taxonomy': 'species', 'res.mass.mean.g.': 'mass'})
con_masses = df[['con.taxonomy', 'con.mass.mean.g.']].dropna().rename(
    columns={'con.taxonomy': 'species', 'con.mass.mean.g.': 'mass'})

# Combine and average duplicate species entries
all_masses = pd.concat([res_masses, con_masses])
species_mass = all_masses.groupby('species')['mass'].mean()

# Fill missing masses with a large number to push them to the end
prey = df['res.taxonomy']
predator = df['con.taxonomy']
species = sorted(set(prey).union(set(predator)))

# species = species[:50]

species_with_mass = pd.Series(species).to_frame(name='species')
species_with_mass['mass'] = species_with_mass['species'].map(species_mass)
species_with_mass['mass'] = species_with_mass['mass'].fillna(np.inf)

# Sort species by ascending mass
species_sorted = species_with_mass.sort_values(by='mass')['species'].tolist()
species_index = {name: i for i, name in enumerate(species_sorted)}
N = len(species_sorted)

# Create adjacency matrix with new order
adj_matrix = np.zeros((N, N), dtype=int)
for res, con in zip(prey, predator):
    if res in species_index and con in species_index:
        i = species_index[res]
        j = species_index[con]
        adj_matrix[i, j] = 1

# Plot reordered adjacency matrix
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(adj_matrix, cmap='Greys', interpolation='none')

ax.set_xticks(range(N))
ax.set_yticks(range(N))
ax.set_xticklabels(species_sorted, rotation=90, fontsize=10)
ax.set_yticklabels(species_sorted, fontsize=10)
ax.set_xlabel("Predator (con.taxonomy)")
ax.set_ylabel("Prey (res.taxonomy)")
ax.set_title("Adjacency Matrix Sorted by Species Mass")
ax.invert_yaxis()

plt.tight_layout()
plt.show()

# Plot Predation Matrix

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load TP, FP, FN CSVs
tp = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_TP_links.csv")
fp = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_FP_links.csv")
fn = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_FN_links.csv")

# Clean NaNs
tp = tp.dropna(subset=['Prey', 'Predator'])
fp = fp.dropna(subset=['Prey', 'Predator'])
fn = fn.dropna(subset=['Prey', 'Predator'])

# Combine species info
all_species = pd.concat([
    tp[['Prey', 'PreyMass']],
    tp[['Predator', 'PredatorMass']].rename(columns={'Predator': 'Prey', 'PredatorMass': 'PreyMass'}),
    fp[['Prey', 'PreyMass']],
    fp[['Predator', 'PredatorMass']].rename(columns={'Predator': 'Prey', 'PredatorMass': 'PreyMass'}),
    fn[['Prey', 'PreyMass']],
    fn[['Predator', 'PredatorMass']].rename(columns={'Predator': 'Prey', 'PredatorMass': 'PreyMass'})
])

# Get unique species sorted by mass
unique_species = all_species.drop_duplicates().sort_values(by='PreyMass').reset_index(drop=True)
species_list = unique_species['Prey'].tolist()
species_index = {name: idx for idx, name in enumerate(species_list)}
N = len(species_list)

# Initialize predation matrix
matrix = [['' for _ in range(N)] for _ in range(N)]

# Fill in the matrix
for df, label in zip([tp, fp, fn], ['TP', 'FP', 'FN']):
    for _, row in df.iterrows():
        prey = row['Prey']
        predator = row['Predator']
        if prey in species_index and predator in species_index:
            i = species_index[prey]
            j = species_index[predator]
            matrix[i][j] = label

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
for i in range(N):
    for j in range(N):
        tag = matrix[i][j]
        if tag == 'TP':
            ax.plot(j, i, 'o', markerfacecolor='black', markeredgecolor='black', markersize=6)
        elif tag == 'FP':
            ax.plot(j, i, 'o', markerfacecolor='none', markeredgecolor='gray', markersize=8)
        elif tag == 'FN':
            ax.plot(j, i, 'o', markerfacecolor='red', markeredgecolor='red', markersize=4)

ax.set_xticks(range(N))
ax.set_yticks(range(N))
ax.set_xticklabels(species_list, rotation=90, fontsize=5)
ax.set_yticklabels(species_list, fontsize=5)
ax.set_xlabel("Predator (sorted by mass)")
ax.set_ylabel("Prey (sorted by mass)")
ax.set_title("Predation Matrix: TP, FP, FN (sorted by mass)")
ax.invert_yaxis()
ax.grid(False)

# Legend
ax.plot([], [], 'o', markerfacecolor='black', markeredgecolor='black', markersize=6, label='TP')
ax.plot([], [], 'o', markerfacecolor='none', markeredgecolor='gray', markersize=8, label='FP')
ax.plot([], [], 'o', markerfacecolor='red', markeredgecolor='red', markersize=4, label='FN')
ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

# Plot Predation Matrix vs Adjacency Matrix (Ground truth)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix

# === Load CSV prediction results ===
tp = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_TP_links.csv").dropna(subset=['Prey', 'Predator'])
fp = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_FP_links.csv").dropna(subset=['Prey', 'Predator'])
fn = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_FN_links.csv").dropna(subset=['Prey', 'Predator'])

# === Load .mat adjacency and metadata ===
mat_data = scipy.io.loadmat("/Users/jorge/Documents/MATLAB/data/foodwebs_mat/lakes_foodweb_tax_mass.mat")
net = mat_data['net']
taxonomy = [str(t[0]) if isinstance(t, (list, np.ndarray)) else str(t) for t in mat_data['taxonomy'].flatten()]
mass = mat_data['mass'].flatten()

# === Sort species by mass ===
taxonomy_df = pd.DataFrame({'Species': taxonomy, 'Mass': mass})
taxonomy_df = taxonomy_df.dropna().sort_values(by='Mass').reset_index(drop=True)
sorted_species = taxonomy_df['Species'].tolist()
species_index = {name: i for i, name in enumerate(sorted_species)}
N = len(sorted_species)

# === Sort adjacency matrix by mass ===
if isinstance(net, coo_matrix):
    net = net.tocsr()
elif isinstance(net, (np.ndarray, list)):
    net = csr_matrix(net)

# Reorder by taxonomy mass order
idx_sorted = [taxonomy.index(sp) for sp in sorted_species]
adj_sorted = net[np.ix_(idx_sorted, idx_sorted)]

# Convert to float dense matrix
adj_sorted_dense = adj_sorted.toarray().astype(float)
adj_sorted_inverted = adj_sorted_dense[::-1, :]  # flip rows only
sorted_species_reversed = sorted_species[::-1]   # for correct tick labels

# === Filter and build predicted matrix ===
tp = tp[tp['Prey'].isin(species_index) & tp['Predator'].isin(species_index)]
fp = fp[fp['Prey'].isin(species_index) & fp['Predator'].isin(species_index)]
fn = fn[fn['Prey'].isin(species_index) & fn['Predator'].isin(species_index)]

pred_matrix = [['' for _ in range(N)] for _ in range(N)]
for df, label in zip([tp, fp, fn], ['TP', 'FP', 'FN']):
    for _, row in df.iterrows():
        i = species_index[row['Prey']]
        j = species_index[row['Predator']]
        pred_matrix[i][j] = label

# === Plotting ===
fig, axs = plt.subplots(1, 2, figsize=(42, 18))

# Left: Predicted Matrix
ax1 = axs[0]
for i in range(N):
    for j in range(N):
        tag = pred_matrix[i][j]
        if tag == 'TP':
            ax1.plot(j, i, 'o', markerfacecolor='black', markeredgecolor='black', markersize=8)
        elif tag == 'FP':
            ax1.plot(j, i, 'o', markerfacecolor='none', markeredgecolor='gray', markersize=10)
        elif tag == 'FN':
            ax1.plot(j, i, 'o', markerfacecolor='red', markeredgecolor='red', markersize=6)

ax1.set_title("Predicted Matrix (TP, FP, FN)")
ax1.set_xticks(range(N))
ax1.set_yticks(range(N))
ax1.set_xticklabels(sorted_species, rotation=90, fontsize=3)
ax1.set_yticklabels(sorted_species, fontsize=3)
ax1.set_xlabel("Predator (mass order)")
ax1.set_ylabel("Prey (mass order)")
ax1.invert_yaxis()
ax1.grid(False)
ax1.plot([], [], 'o', markerfacecolor='black', markeredgecolor='black', markersize=8, label='TP')
ax1.plot([], [], 'o', markerfacecolor='none', markeredgecolor='gray', markersize=10, label='FP')
ax1.plot([], [], 'o', markerfacecolor='red', markeredgecolor='red', markersize=6, label='FN')
ax1.legend(loc='upper right', fontsize=8)

# Right: Adjacency Matrix
ax2 = axs[1]
ax2.imshow(adj_sorted_inverted, cmap='Greys', interpolation='none')
ax2.set_title("Original Adjacency Matrix (Corrected Prey Order)")
ax2.set_xticks(range(N))
ax2.set_yticks(range(N))
ax2.set_xticklabels(sorted_species, rotation=90, fontsize=3)
ax2.set_yticklabels(sorted_species_reversed, fontsize=3)
ax2.set_xlabel("Predator (mass order)")
ax2.set_ylabel("Prey (mass order)")
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix

# === Load CSV prediction results ===
tp = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/Chesapeake Bay_tax_mass_K_10_TP_links.csv").dropna(subset=['Prey', 'Predator'])
fp = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/Chesapeake Bay_tax_mass_K_10_FP_links.csv").dropna(subset=['Prey', 'Predator'])
fn = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/Chesapeake Bay_tax_mass_K_10_FN_links.csv").dropna(subset=['Prey', 'Predator'])

# === Load .mat adjacency and metadata ===
mat_data = scipy.io.loadmat("/Users/jorge/Documents/MATLAB/data/foodwebs_mat/Chesapeake Bay_tax_mass.mat")
net = mat_data['net']
taxonomy = [str(t[0]) if isinstance(t, (list, np.ndarray)) else str(t) for t in mat_data['taxonomy'].flatten()]
mass = mat_data['mass'].flatten()

# === Sort species by mass ===
taxonomy_df = pd.DataFrame({'Species': taxonomy, 'Mass': mass})
taxonomy_df = taxonomy_df.dropna().sort_values(by='Mass').reset_index(drop=True)
sorted_species = taxonomy_df['Species'].tolist()
species_index = {name: i for i, name in enumerate(sorted_species)}
N = len(sorted_species)

# === Sort adjacency matrix by mass ===
if isinstance(net, coo_matrix):
    net = net.tocsr()
elif isinstance(net, (np.ndarray, list)):
    net = csr_matrix(net)

# Reorder by taxonomy mass order
idx_sorted = [taxonomy.index(sp) for sp in sorted_species]
adj_sorted = net[np.ix_(idx_sorted, idx_sorted)]

# Convert to float dense matrix
adj_sorted_dense = adj_sorted.toarray().astype(float)
adj_sorted_inverted = adj_sorted_dense[::-1, :]  # flip rows only
sorted_species_reversed = sorted_species[::-1]   # for correct tick labels

# === Filter and build predicted matrix ===
tp = tp[tp['Prey'].isin(species_index) & tp['Predator'].isin(species_index)]
fp = fp[fp['Prey'].isin(species_index) & fp['Predator'].isin(species_index)]
fn = fn[fn['Prey'].isin(species_index) & fn['Predator'].isin(species_index)]

pred_matrix = [['' for _ in range(N)] for _ in range(N)]
for df, label in zip([tp, fp, fn], ['TP', 'FP', 'FN']):
    for _, row in df.iterrows():
        i = species_index[row['Prey']]
        j = species_index[row['Predator']]
        pred_matrix[i][j] = label

# === Plotting ===
fig, axs = plt.subplots(1, 2, figsize=(42, 18))

# Left: Predicted Matrix
ax1 = axs[0]
for i in range(N):
    for j in range(N):
        tag = pred_matrix[i][j]
        if tag == 'TP':
            ax1.plot(j, i, 'o', markerfacecolor='black', markeredgecolor='black', markersize=8)
        elif tag == 'FP':
            ax1.plot(j, i, 'o', markerfacecolor='none', markeredgecolor='gray', markersize=10)
        elif tag == 'FN':
            ax1.plot(j, i, 'o', markerfacecolor='red', markeredgecolor='red', markersize=6)

ax1.set_title("Predicted Matrix (TP, FP, FN)")
ax1.set_xticks(range(N))
ax1.set_yticks(range(N))
ax1.set_xticklabels(sorted_species, rotation=90, fontsize=3)
ax1.set_yticklabels(sorted_species, fontsize=3)
ax1.set_xlabel("Predator (mass order)")
ax1.set_ylabel("Prey (mass order)")
ax1.invert_yaxis()
ax1.grid(False)
ax1.plot([], [], 'o', markerfacecolor='black', markeredgecolor='black', markersize=8, label='TP')
ax1.plot([], [], 'o', markerfacecolor='none', markeredgecolor='gray', markersize=10, label='FP')
ax1.plot([], [], 'o', markerfacecolor='red', markeredgecolor='red', markersize=6, label='FN')
ax1.legend(loc='upper right', fontsize=8)

# Right: Adjacency Matrix
ax2 = axs[1]
ax2.imshow(adj_sorted_inverted, cmap='Greys', interpolation='none')
ax2.set_title("Original Adjacency Matrix (Corrected Prey Order)")
ax2.set_xticks(range(N))
ax2.set_yticks(range(N))
ax2.set_xticklabels(sorted_species, rotation=90, fontsize=3)
ax2.set_yticklabels(sorted_species_reversed, fontsize=3)
ax2.set_xlabel("Predator (mass order)")
ax2.set_ylabel("Prey (mass order)")
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

# Precision-Recall Curve

In [ ]:
import pandas as pd
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

df = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_scores_labels.csv")
precision, recall, thresholds = precision_recall_curve(df['Label'], df['Score'])

plt.figure(figsize=(6, 6))
plt.plot(recall, precision, marker='o', color='orange')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

df = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/Chesapeake Bay_tax_mass_K_10_scores_labels.csv")
precision, recall, thresholds = precision_recall_curve(df['Label'], df['Score'])

plt.figure(figsize=(6, 6))
plt.plot(recall, precision, marker='o', color='red')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.grid(True)
plt.show()

# (Receiver Operating Characteristic) ROC Curve

In [ ]:
import pandas as pd
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Load the scores and true labels
df = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/lakes_foodweb_tax_mass_K_10_scores_labels.csv")
y_true = df['Label']
y_score = df['Score']

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

# Plot
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # chance line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Load the scores and true labels
df = pd.read_csv("/Users/jorge/Documents/MATLAB/data/result/confusion_matrix_csv/Chesapeake Bay_tax_mass_K_10_scores_labels.csv")
y_true = df['Label']
y_score = df['Score']

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

# Plot
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='red', lw=2, label=f"ROC curve (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # chance line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()